In [ ]:
import keras
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.optimizers import Adam
import math
import numpy as np
import random
from collections import deque #Doubly Ended Queue

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
class Agent():
  def __init__(self, state_size, is_eval=False, model_name=""):
    self.state_size = state_size # normalized previous days
    self.action_size = 3 # sit, buy, sell
    self.memory = deque(maxlen=1000)
    self.inventory = []
    self.model_name = model_name
    self.is_eval = is_eval
    self.gamma = 0.95
    self.epsilon = 1.0
    self.epsilon_min = 0.01
    self.epsilon_decay = 0.995
    self.model = load_model(model_name) if is_eval else self._model()
  def _model(self):
    model = Sequential()
    model.add(Dense(units=64, input_dim=self.state_size, activation="relu"))
    model.add(Dense(units=32, activation="relu"))
    model.add(Dense(units=8, activation="relu"))
    model.add(Dense(self.action_size, activation="linear"))
    model.compile(loss="mse", optimizer=Adam(lr=0.001))
    return model
  def act(self, state):
    if not self.is_eval and random.random()<= self.epsilon:
      return random.randrange(self.action_size)
    options = self.model.predict(state)
    return np.argmax(options[0])
  def expReplay(self, batch_size):
    mini_batch = []
    l = len(self.memory)
    for i in range(l - batch_size + 1, l):
      mini_batch.append(self.memory[i])
    for state, action, reward, next_state, done in mini_batch:
      target = reward
    if not done:
      target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
    target_f = self.model.predict(state)
    target_f[0][action] = target
    self.model.fit(state, target_f, epochs=1, verbose=0)
    if self.epsilon > self.epsilon_min:
      self.epsilon *= self.epsilon_decay

In [ ]:
def formatPrice(n):
    return("-Rs." if n<0 else "Rs.")+"{0:.2f}".format(abs(n))
def getStockDataVec(key):
    vec = []
    lines = open(key+".csv","r").read().splitlines()
    for line in lines[1:]:
        vec.append(float(line.split(",")[5]))
    return vec 
def sigmoid(x):
    return 1/(1+math.exp(-x))
def getState(data, t, n):
    d = t - n + 1
    block = data[d:t + 1] if d >= 0 else -d * [data[0]] + data[0:t + 1] # pad with t0
    res = []
    for i in range(n - 1):
        res.append(sigmoid(block[i + 1] - block[i]))
    return np.array([res])

In [ ]:
k1=10 #MAX_TRANSACTION
k2=10000 #TOTAL_MONEY
k3=0 #CURRENT_STOCKS_BOUGHT
k4=0 #CURRENT_TRANSACTION_COUNT

In [ ]:
import sys
stock_name = input("Enter stock_name, window_size, Episode_count")
window_size = input()
episode_count = input()
stock_name = str(stock_name)
window_size = int(window_size)
episode_count = int(episode_count)
agent = Agent(window_size)
data = getStockDataVec(stock_name)
l = len(data) - 1
batch_size = 32
for e in range(episode_count):
  print("Episode " + str(e+1) + "/" + str(episode_count))
  state = getState(data, 0, window_size + 1)
  total_profit = 0
  agent.inventory = []
  for t in range(l):
      action = agent.act(state)
      # sit
      next_state = getState(data, t + 1, window_size + 1)
      reward = 0
      if action == 1: # buy 
          if k4<k1 and k2>=data[t]:
            k4 = k4+1
            k2 = k2-data[t]
            agent.inventory.append(data[t])
            print("Buy: " + formatPrice(data[t]))
      elif action == 2 and len(agent.inventory) > 0: # sell
          k4=0
          k2 = k2+data[t]
          bought_price = window_size_price = agent.inventory.pop(0)
          reward = max(data[t] - bought_price)
          total_profit += data[t] - bought_price
          print("Sell: " + formatPrice(data[t]) + " | Profit: " + formatPrice(data[t] - bought_price))
      done = True if t == l - 1 else False
      agent.memory.append((state, action, reward, next_state, done))
      state = next_state
      if done:
          print("--------------------------------")
          print("Total Profit: " + formatPrice(total_profit))
          print("--------------------------------")
      if len(agent.memory) > batch_size:
          agent.expReplay(batch_size)
  agent.model.save("gdrive/MyDrive/models/tatamotors"+str(e+1))
  

Enter stock_name, window_size, Episode_countTATAMOTORS.NS_train
10
10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Episode 1/10
Buy: Rs.27.52
Buy: Rs.27.94
Buy: Rs.27.42
Sell: Rs.26.62 | Profit: -Rs.0.90
Buy: Rs.24.28
Buy: Rs.24.67
Buy: Rs.25.64
Sell: Rs.26.79 | Profit: -Rs.1.15
Sell: Rs.27.34 | Profit: -Rs.0.08
Buy: Rs.26.26
Sell: Rs.24.33 | Profit: Rs.0.05
Buy: Rs.23.95
Buy: Rs.25.86
Sell: Rs.25.24 | Profit: Rs.0.58
Sell: Rs.25.16 | Profit: -Rs.0.47
Sell: Rs.24.35 | Profit: -Rs.1.91
Sell: Rs.24.58 | Profit: Rs.0.63
Sell: Rs.24.43 | Profit: -Rs.1.43
Buy: Rs.24.13
Sell: Rs.25.59 | Profit: Rs.1.46
Buy: Rs.25.91
Sell: Rs.26.14 | Profit: Rs.0.23
Buy: Rs.25.33
Sell: Rs.24.96 | Profit: -Rs.0.37
Buy: Rs.26.68
Buy: Rs.29.54
Sell: Rs.30.45 | Profit: Rs.3.77
Sell: Rs.32.22 | Profit: Rs.2.69
Buy: Rs.29.39
Sell: Rs.30.35 | Profit: Rs.0.96
Buy: Rs.29.67
Sell: Rs.32.13 | Profit: Rs.2.46
Buy: Rs.37.20
Buy: Rs.41.43
Sell: Rs.46.06 | Profit: Rs.8.86
Buy: Rs.44.10
Sell: Rs.43.07 | Profit: Rs.1.64
Sell: Rs.42.62 | Profit: -Rs.1.49
Buy: Rs.44.83
Sell: Rs.44.53 | Profit: -Rs.0.29
Buy: Rs.49.57
Sell: Rs.48.38 | Profit:

In [ ]:
for i in range(10):
  initial_amount=10000
  print("initial_amount:",initial_amount)
  amount=10000
  #stock_name = input("Enter Stock_name, Model_name")
  stock_name="TATAMOTORS.NS_eval"
  model_name = "/content/gdrive/MyDrive/models/tatamotors"+str(i+1)
  #model_name = input()
  model = load_model(model_name)
  window_size = model.layers[0].input.shape.as_list()[1]
  agent = Agent(window_size, True, model_name)
  data = getStockDataVec(stock_name)
  #print(data)
  l = len(data) - 1
  batch_size = 32
  state = getState(data, 0, window_size + 1)
  #print(state)
  total_profit = 0
  agent.inventory = [] 
  n_stocks = []
  #print(l)
  for t in range(l):
      action = agent.act(state)
      print(action)
      # sit
      next_state = getState(data, t + 1, window_size + 1)
      reward = 0
      if action == 1: # buy
          if(amount>=data[t]):
            n_stocks.append(int(amount/data[t]))
            amount=amount-int(amount/data[t])*data[t]
            agent.inventory.append(data[t])
            #print("Buy: " + formatPrice(data[t]))
      elif action == 2 and len(agent.inventory) > 0: # sell

          bought_price = agent.inventory.pop(0)
          n_stock = n_stocks.pop(0)
          reward = max(data[t] - bought_price, 0)
          total_profit += (data[t] - bought_price)*n_stock
          amount=amount+n_stock*data[t]
          #print("Sell: " + formatPrice(data[t]) + " | Profit: " + formatPrice((data[t] - bought_price)*n_stock))
      done = True if t == l - 1 else False
      agent.memory.append((state, action, reward, next_state, done))
      state = next_state
      if done:
          print("--------------------------------")
          #print(stock_name + " Total Profit: " + formatPrice(total_profit))
          print("--------------------------------")
          #print ("Total profit is:",formatPrice(total_profit))
          #print("final_amount:",amount)
          print("Profit(in %) for episode "+str(i+1)+" =",total_profit*100/initial_amount )


In [ ]:
initial_amount=10000
print("initial_amount:",initial_amount)
amount=10000
stock_name="TATAMOTORS.NS19_eval"
model_name = "/content/gdrive/MyDrive/models/tatamotors8"
model = load_model(model_name)
window_size = model.layers[0].input.shape.as_list()[1]
agent = Agent(window_size, True, model_name)
data = getStockDataVec(stock_name)
l = len(data) - 1
batch_size = 32
state = getState(data, 0, window_size + 1)
total_profit = 0
agent.inventory = [] 
n_stocks = []
for t in range(l):
    action = agent.act(state)
    #print(action)
    next_state = getState(data, t + 1, window_size + 1)
    reward = 0
    if action == 1: # buy
        if(amount>=data[t]):
          n_stocks.append(int(amount/data[t]))
          amount=amount-int(amount/data[t])*data[t]
          agent.inventory.append(data[t])
          print("Buy: " + formatPrice(data[t]))
    elif action == 2 and len(agent.inventory) > 0: # sell
        bought_price = agent.inventory.pop(0)
        n_stock = n_stocks.pop(0)
        reward = max(data[t] - bought_price, 0)
        total_profit += (data[t] - bought_price)*n_stock
        amount=amount+n_stock*data[t]
        print("Sell: " + formatPrice(data[t]) + " | Profit: " + formatPrice((data[t] - bought_price)*n_stock))
    done = True if t == l - 1 else False
    agent.memory.append((state, action, reward, next_state, done))
    state = next_state
    if done:
        print("--------------------------------")
        print(stock_name + " Total Profit: " + formatPrice(total_profit))
        print("--------------------------------")
        print("Profit(in %) =",total_profit*100/initial_amount )

initial_amount: 10000
Buy: Rs.180.30
Sell: Rs.182.40 | Profit: Rs.115.50
Buy: Rs.151.80
Sell: Rs.159.85 | Profit: Rs.531.30
Buy: Rs.177.35
Sell: Rs.177.45 | Profit: Rs.6.00
Buy: Rs.215.95
Sell: Rs.231.75 | Profit: Rs.774.20
Buy: Rs.230.50
Sell: Rs.236.25 | Profit: Rs.281.75
Buy: Rs.176.85
Sell: Rs.190.15 | Profit: Rs.877.80
Buy: Rs.175.15
Sell: Rs.182.30 | Profit: Rs.507.65
Buy: Rs.169.05
Sell: Rs.167.40 | Profit: -Rs.127.05
Buy: Rs.162.55
Sell: Rs.168.00 | Profit: Rs.430.55
Buy: Rs.163.75
Sell: Rs.162.30 | Profit: -Rs.117.45
Buy: Rs.167.90
Sell: Rs.160.75 | Profit: -Rs.564.85
Buy: Rs.154.85
Sell: Rs.156.75 | Profit: Rs.155.80
Buy: Rs.123.75
Sell: Rs.122.50 | Profit: -Rs.130.00
Buy: Rs.112.65
Sell: Rs.109.50 | Profit: -Rs.355.95
Buy: Rs.128.60
Sell: Rs.122.00 | Profit: -Rs.633.60
Buy: Rs.123.05
Sell: Rs.124.50 | Profit: Rs.137.75
Buy: Rs.127.85
Sell: Rs.126.95 | Profit: -Rs.82.80
Buy: Rs.175.05
Sell: Rs.172.20 | Profit: -Rs.190.95
Buy: Rs.172.00
Sell: Rs.174.50 | Profit: Rs.167.50
Buy: